
# DS1 Spark + Python Prep Notebook

This notebook is designed as a **comprehensive but focused** preparation for a **Data Scientist 1** interview that uses:

- PySpark (DataFrames)
- SQL-style data work
- Basic Python

It assumes:
- You have a `SparkSession` named `spark` (Databricks / EMR / local)
- You are working with a CSV-like dataset (e.g., retail transactions)

You do **not** need deep ML or graph databases for this prep. The emphasis is on **data manipulation, querying, and reasoning**.



## 1. Environment Setup

In Databricks or most managed Spark environments, a `SparkSession` named `spark` is already available.

If you're running this locally in Jupyter, you may need to create it.


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, instr, lit, when,
    count, sum as _sum, avg, max as _max, min as _min,
    row_number, rank, dense_rank, desc, asc
)
from pyspark.sql.window import Window

# If Spark is not yet created (e.g., local Jupyter), uncomment:
# spark = SparkSession.builder #     .appName("ds1_spark_prep_notebook") #     .getOrCreate()

spark



## 2. Loading Data as a DataFrame

**Key interview idea:**  
- A CSV is a **file on disk** (raw bytes).  
- A **DataFrame** is a **structured, distributed table** in Spark, created *after* reading the file.

Below, adjust the path to point to a CSV on your system. If you have the Spark Definitive Guide data, you can use something like `data/retail-data/by-day/2010-12-01.csv`.


In [ ]:

# TODO: Update this path to a real CSV on your system.
csv_path = "data/retail-data/by-day/2010-12-01.csv"

df = spark.read.csv(
    csv_path,
    header=True,
    inferSchema=True
)

df.printSchema()
df.show(5, truncate=False)



## 3. Basic DataFrame Operations

Core skills:
- `select` specific columns
- Create new columns with `withColumn`
- Filter rows with `filter` / `where`


In [ ]:

# Select a subset of columns
df.select("InvoiceNo", "StockCode", "Description", "Quantity", "UnitPrice").show(5, truncate=False)

# Add a computed column: TotalCost = UnitPrice * Quantity
df_with_total = df.withColumn("TotalCost", col("UnitPrice") * col("Quantity"))
df_with_total.select("InvoiceNo", "StockCode", "Quantity", "UnitPrice", "TotalCost").show(5, truncate=False)

# Filter: UnitPrice greater than 10
df_with_total.filter(col("UnitPrice") > 10).select("InvoiceNo", "UnitPrice").show(5, truncate=False)



## 4. Boolean Filters and String Matching

This section mirrors the pattern you remember:

```python
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()
```


In [ ]:

from pyspark.sql.functions import instr

# Boolean conditions
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1

# Chain where() conditions and use boolean OR
filtered = df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter)

filtered.select("InvoiceNo", "StockCode", "Description", "UnitPrice").show(20, truncate=False)



## 5. Handling NULLs

Common tasks:
- Count NULLs
- Drop rows with critical NULLs
- Fill NULLs with default values


In [ ]:

# Count rows where Description is NULL
df.select(
    count(when(col("Description").isNull(), 1)).alias("NullDescriptionCount")
).show()

# Drop rows where key columns are NULL
df_clean = df.dropna(subset=["InvoiceNo", "StockCode", "Quantity", "UnitPrice"])
print("Original count:", df.count())
print("After dropna count:", df_clean.count())

# Fill NULL descriptions with a default label
df_filled = df_clean.fillna({"Description": "UNKNOWN"})
df_filled.select("InvoiceNo", "StockCode", "Description").show(5, truncate=False)



## 6. Aggregations with `groupBy` and `agg`

Core interview operations:
- `groupBy` one or more keys
- `agg` with functions like `sum`, `avg`, `count`
- `orderBy` results


In [ ]:

df_with_total = df.withColumn("TotalCost", col("UnitPrice") * col("Quantity"))

agg_by_stock = df_with_total.groupBy("StockCode").agg(
    _sum("Quantity").alias("TotalQuantity"),
    _sum("TotalCost").alias("TotalRevenue"),
    avg("UnitPrice").alias("AvgUnitPrice")
)

agg_by_stock.orderBy(desc("TotalRevenue")).show(10, truncate=False)



### 6.1 Multiple Grouping Keys

You can group by multiple columns, e.g., `CustomerID` + `Country`.


In [ ]:

# Example assuming df has CustomerID and Country (some versions of the retail dataset do)
group_cols = [c for c in df.columns if c in ["CustomerID", "Country"]]
print("Possible group columns detected:", group_cols)

if "CustomerID" in group_cols and "Country" in group_cols:
    agg_by_customer_country = df_with_total.groupBy("CustomerID", "Country").agg(
        _sum("TotalCost").alias("TotalSpend")
    )
    agg_by_customer_country.orderBy(desc("TotalSpend")).show(10, truncate=False)
else:
    print("CustomerID / Country not found in this dataset; adjust for your schema.")



## 7. Joins

Key join concepts they might ask:
- INNER vs LEFT joins
- Joining on a key column
- Understanding row explosion / duplication

We’ll create small example DataFrames to practice the pattern.


In [ ]:

# Small example: customers and countries
customers = spark.createDataFrame(
    [
        (1, "C001", "UK"),
        (2, "C002", "Germany"),
        (3, "C003", "France"),
    ],
    ["CustomerPK", "CustomerID", "CountryCode"]
)

countries = spark.createDataFrame(
    [
        ("UK", "United Kingdom"),
        ("Germany", "Germany"),
        ("France", "France"),
    ],
    ["CountryCode", "CountryName"]
)

customers.show()
countries.show()

# LEFT join on CountryCode
customers_with_country = customers.join(
    countries,
    on="CountryCode",
    how="left"
)

customers_with_country.show()



### 7.1 Joining with a Fact Table

Pattern:
1. Aggregate a fact table (e.g., transactions by `CustomerID`)
2. Join with a dimension (e.g., customer master or country table)


In [ ]:

# Example: total spend per CustomerID from main df
if "CustomerID" in df.columns:
    df_with_total = df.withColumn("TotalCost", col("UnitPrice") * col("Quantity"))

    customer_spend = df_with_total.groupBy("CustomerID").agg(
        _sum("TotalCost").alias("TotalSpend")
    )

    # Join with customers table (on CustomerID)
    customer_spend_joined = customer_spend.join(
        customers,
        on="CustomerID",
        how="left"
    )

    customer_spend_joined.orderBy(desc("TotalSpend")).show(10, truncate=False)
else:
    print("CustomerID not found in df; adjust join logic for your real schema.")



## 8. Window Functions (Ranking Within Groups)

Common pattern: **“For each X, find the top Y by metric M.”**  
Example: For each customer, find their most expensive purchase.

Tools:
- `Window.partitionBy(...).orderBy(...)`
- `row_number`, `rank`, `dense_rank`


In [ ]:

df_with_total = df.withColumn("TotalCost", col("UnitPrice") * col("Quantity"))

if "CustomerID" in df.columns:
    window_spec = Window.partitionBy("CustomerID").orderBy(desc("TotalCost"))

    df_ranked = df_with_total.withColumn("rn", row_number().over(window_spec))

    top_purchase_per_customer = df_ranked.where(col("rn") == 1)

    top_purchase_per_customer.select("CustomerID", "InvoiceNo", "TotalCost").show(10, truncate=False)
else:
    print("CustomerID not found in df; window example limited to global ranking.")

# Global top-N invoices by TotalCost
window_spec_global = Window.orderBy(desc("TotalCost"))
df_ranked_global = df_with_total.withColumn("rn_global", row_number().over(window_spec_global))
df_ranked_global.select("InvoiceNo", "TotalCost", "rn_global").where(col("rn_global") <= 10).show(truncate=False)



## 9. Spark SQL: Using Temp Views

Spark lets you switch between:
- DataFrame API (methods like `select`, `filter`, `groupBy`)
- SQL (strings passed to `spark.sql`)

They compile to the same engine underneath.


In [ ]:

df_with_total.createOrReplaceTempView("purchases")

top_customers_sql = spark.sql("""
    SELECT
        CustomerID,
        SUM(UnitPrice * Quantity) AS TotalSpend
    FROM purchases
    WHERE CustomerID IS NOT NULL
    GROUP BY CustomerID
    ORDER BY TotalSpend DESC
    LIMIT 10
""")

top_customers_sql.show()



## 10. Data Cleansing with `when` / `otherwise`

Example tasks:
- Flagging suspicious rows
- Creating category labels
- Replacing out-of-range values


In [ ]:

# Example: create a flag column for high-value lines
df_flags = df_with_total.withColumn(
    "HighValueFlag",
    when(col("TotalCost") > 1000, lit(1)).otherwise(lit(0))
)

df_flags.select("InvoiceNo", "TotalCost", "HighValueFlag").orderBy(desc("TotalCost")).show(10, truncate=False)



## 11. User-Defined Functions (UDFs) – Optional

You typically try to stay with **built-in** functions for performance, but it’s good to know UDFs exist.

(For DS1, you can mostly just say that UDFs allow custom Python logic when built-ins are not enough, but they can be slower.)


In [ ]:

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def categorize_price(unit_price):
    if unit_price is None:
        return "UNKNOWN"
    if unit_price < 1:
        return "cheap"
    elif unit_price < 10:
        return "moderate"
    else:
        return "expensive"

categorize_price_udf = udf(categorize_price, StringType())

df_price_cat = df.withColumn("PriceCategory", categorize_price_udf(col("UnitPrice")))
df_price_cat.select("UnitPrice", "PriceCategory").show(10, truncate=False)



## 12. Python Refresher (Likely Coding Level for DS1)

This is about comfort with:
- lists
- dicts
- comprehensions
- simple functions


In [ ]:

# Lists and comprehensions
nums = [1, 2, 3, 4, 5, 6]

even_nums = [n for n in nums if n % 2 == 0]
squared = [n * n for n in nums]

print("Even numbers:", even_nums)
print("Squared numbers:", squared)

# Dictionaries
prices = {"apple": 1.0, "banana": 0.5, "orange": 0.8}
prices_with_tax = {k: v * 1.1 for k, v in prices.items()}
print("Prices with tax:", prices_with_tax)

# Simple utility function
def total_price(items, price_lookup):
    total = 0.0
    for item in items:
        total += price_lookup.get(item, 0.0)
    return total

basket = ["apple", "banana", "banana"]
print("Basket:", basket)
print("Total price:", total_price(basket, prices))



## 13. High-Level ML Workflow (Verbal Understanding Only)

For DS1, you generally just need to **describe** the supervised learning process:

1. **Define the problem**  
   - Regression (predict a number) or classification (predict a label).

2. **Prepare the data**  
   - Select features and label.  
   - Handle missing values.  
   - Encode categoricals if needed.  
   - Scale/normalize if appropriate.

3. **Split into train and test sets**  
   - e.g., 70/30 or 80/20.

4. **Choose a model**  
   - e.g., logistic regression, random forest, gradient boosted trees.

5. **Train the model on the training set**.

6. **Evaluate on the test set**  
   - Metrics: accuracy, precision/recall, ROC AUC, RMSE, etc.

7. **Iterate**  
   - Try new features, different models, tuning hyperparameters.

You usually do **not** need to derive formulas or gradients for DS1 – just show you understand the pipeline.



## 14. What to Be Ready to Explain in an Interview

If you can confidently talk through and sketch code for:

- How to read a CSV into a DataFrame
- How to inspect schema and data
- How to `select`, `filter`, `withColumn`
- How to aggregate with `groupBy().agg()`
- How to `join` two DataFrames
- What a window function is used for
- How to register a temp view and query with SQL
- How to handle NULLs and create flags with `when`
- Basic Python list/dict/loop skills
- High-level ML workflow

…then you are in **very solid shape** for a DS1 interview.

Use this notebook as your **sandbox**: run things, break things, and fix them. That process is exactly what real work looks like.
